# Train an LSTM based controller 

Train and save an LSTM-based controller. It contains:
* Code for loading and pre-processing the training data. 
* Training an LSTM with specific parameters and saving it

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
#from pprint import pformat


#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

from sensorprocessing import sp_conv_vae
from demo_to_trainingdata import create_RNN_training_sequence_xy, BCDemonstration

Loading pointer config file: /home/lboloni/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/lboloni/Insync/lotzi.boloni@gmail.com/Google Drive/LotziStudy/Code/PackageTracking/BerryPicker/settings/settings-tredy2.yaml


### Creating training data

In it's current form it creates sequential data from a single demonstration. 

TODO: this needs to create the training data from a mixture of various demonstrations

In [2]:
task = "proprioception-uncluttered"
sp = sp_conv_vae.ConvVaeSensorProcessing()

demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)

inputlist = []
targetlist = []

for demo_dir in task_dir.iterdir():
    if not demo_dir.is_dir():
        pass
    bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
    print(bcd)
    z, a = bcd.read_z_a()
    print(z.shape)
    print(a.shape)
    inputs, targets = create_RNN_training_sequence_xy(z, a, sequence_length=10)
    inputlist.append(inputs)
    targetlist.append(targets)

inputs = torch.cat(inputlist)
targets = torch.cat(targetlist)


resume_model and jsonfile are:
	resume_model=/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-models/Conv-VAE/models/VAE_Robot/0901_125042/checkpoint-epoch171.pth
	jsonfile=/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-models/Conv-VAE/models/VAE_Robot/0901_125042/config.json
{
    "name": "VAE_Robot",
    "n_gpu": 1,
    "arch": {
        "type": "VanillaVAE",
        "args": {
            "in_channels": 3,
            "latent_dims": 128,
            "flow": false
        }
    },
    "data_loader": {
        "###type-prev": "RobotDataLoader",
        "type": "CelebDataLoader",
        "args": {
            "data_dir": "/home/lboloni/Documents/Hackingwork/__Temporary/VisionBasedRobotManipulator-training-data/vae-training-data",
            "batch_size": 64,
            "shuffle": true,
            "validation_split": 0.2,
            "num_workers": 2
        }
    },
    "optimizer": {
        "type": "Adam",
        "args": {
            "lr": 0.005,
         

/home/lboloni/Documents/Hackingwork/_Checkouts/BerryPicker/BerryPicker/src/behavior_cloning/../sensorprocessing/sp_conv_vae.py:72: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature

Cameras found: ['dev2']
There are 753 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 753,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x78aa2c2aa830>,
 'source_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-demos/demos/proprioception-uncluttered/2024_10_26__16_31_40'),
 'trim_from': 1,
 'trim_to': 753}
(752, 128)
(752, 6)
Cameras found: ['dev2']
There are 968 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
{'actiontype': 'rc-position-target',
 'camera': 'dev2',
 'cameras': ['dev2'],
 'maxsteps': 968,
 'sensorprocessor': <sensorprocessing.sp_conv_vae.ConvVaeSensorProcessing object at 0x78aa2c2aa830>,
 'source_dir': PosixPath('/home/lboloni/Documents/Hackingwork/__Temporary/BerryPicker-demos/demos/proprioception-uncluttered/2024_10_26__

## LSTM w/ 3 layers and skip connections

This is an attempt to recreate the LSTM model from the Rouhollah 2020 paper. It does not have an MDM at the end. 

In [3]:
# This is the architecture created by chatgpt
class LSTMXYPredictor(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size, num_layers):
        super(LSTMXYPredictor, self).__init__()
        self.lstm = nn.LSTM(latent_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [batch_size, sequence_length, latent_size]
        out, _ = self.lstm(x)  # LSTM output shape: [batch_size, sequence_length, hidden_size]
        out = self.fc(out[:, -1, :])  # Take last time step output and pass through the fully connected layer
        return out  # Predicted next vector

In [4]:
# Original
latent_size = Config()["robot"]["latent_encoding_size"]  
hidden_size = 32  # degrees of freedom in the robot
output_size = 6  # degrees of freedom in the robot
num_layers = 2

# Simple model: Instantiate model, loss function, and optimizer
#model = LSTMResidualController(latent_size=latent_size, hidden_size=hidden_size)
#criterion = nn.MSELoss()  # Mean Squared Error for regression
#optimizer = optim.Adam(model.parameters(), lr=0.001)


# Instantiate model, loss function, and optimizer
model = LSTMXYPredictor(latent_size=latent_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
z

array([[ 0.07089406, -0.32879138,  0.3921941 , ...,  0.0507906 ,
        -0.11549067, -0.4683873 ],
       [ 0.07325773, -0.32755402,  0.39171767, ...,  0.05148034,
        -0.11586552, -0.46543336],
       [ 0.06882108, -0.32531896,  0.38906685, ...,  0.05078246,
        -0.11425927, -0.45738766],
       ...,
       [ 0.03610518, -0.33893436,  0.3692953 , ...,  0.0880032 ,
        -0.13571778, -0.497684  ],
       [ 0.03718238, -0.3391816 ,  0.3708424 , ...,  0.08734815,
        -0.13582218, -0.50285035],
       [ 0.03575388, -0.33898264,  0.36980066, ...,  0.08724476,
        -0.13428041, -0.49851295]], dtype=float32)

In [ ]:
# Training Loop
num_epochs = 100

z, a = bcd.read_z_a()
num_sequences = inputs.shape[0]

for epoch in range(num_epochs):
    model.train()
    
    # Loop over each sequence in the batch
    for i in range(num_sequences):
        # Prepare input and target
        input_seq = inputs[i]
        target = targets[i]

        # Reshape for batch compatibility
        input_seq = input_seq.unsqueeze(0)  # Shape: [1, sequence_length, latent_size]
        target = target.unsqueeze(0)        # Shape: [1, latent_size]

        # Forward pass
        output = model(input_seq)
        loss = criterion(output, target)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 2 == 0: # was 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")

# FIXME: save the model
filename_lstm = Config()["controller"]["lstm_model_file"]
torch.save(model.state_dict(), filename_lstm)

Epoch [2/100], Loss: 90.2824
Epoch [4/100], Loss: 56.6518
Epoch [6/100], Loss: 38.6657
Epoch [8/100], Loss: 51.7366
Epoch [10/100], Loss: 25.9904
Epoch [12/100], Loss: 40.4824
Epoch [14/100], Loss: 42.6599
Epoch [16/100], Loss: 8.3470
Epoch [18/100], Loss: 18.6223
Epoch [20/100], Loss: 52.5237
Epoch [22/100], Loss: 14.5628
